Bu hücrede, *pandas* kütüphanesi `pd` olarak içe aktarılıyor. Daha sonra, `pd.read_excel` fonksiyonu kullanılarak `"./fiat_cars.xlsx"` dosyasından veri yükleniyor. `index_col=0` parametresi, verilerin ilk sütununun satır indeksleri olarak kullanılmasını sağlıyor. Bu sayede veriler daha düzenli bir şekilde işlenebilir.

In [1]:
import pandas as pd
data = pd.read_excel("./fiat_cars_raw.xlsx", index_col=0)

Bu hücrede, `data` veri çerçevesinin sütun adları daha açıklayıcı olacak şekilde yeniden adlandırılıyor. `data.rename` fonksiyonu, sütunların eski isimlerini, yeni isimlerle değiştiren bir sözlük (`columns` parametresi) kullanıyor. Örneğin, "Motor Hacmi" sütunu "Motor Hacmi (CC)" olarak güncelleniyor. Bu işlem, birim bilgilerini ekleyerek sütunları daha anlaşılır hale getiriyor ve verilerin okunabilirliğini artırıyor.

In [ ]:
data = data.rename(
  columns = {
    'Motor Hacmi': 'Motor Hacmi (CC)',
    'Motor Gücü': 'Motor Gücü (HP)',
    'Ort. Yakıt Tüketimi': 'Ort. Yakıt Tüketimi (Litre)',
    'Yakıt Deposu': 'Yakıt Deposu (Litre)',
    'Price': 'Price (TL)'
  }
)

Bu hücrede, veri çerçevesindeki eksik değerlere sahip sütunlar analiz edilip belirli kriterlere göre siliniyor:

1. İlk satırda, `data.isnull().sum()` ile her sütundaki eksik değer sayısı hesaplanıyor ve `data_set_null_count` adlı bir sözlük olarak kaydediliyor.
2. `columns_to_drop` listesi, `data_set_null_count` sözlüğündeki eksik değeri 100'den fazla olan sütun adlarıyla dolduruluyor. Bu işlem, çok fazla eksik değere sahip sütunların belirlenmesi için yapılır.
3. Ardından, `Ort. Yakıt Tüketimi (Litre)` sütunu, bu listeden elle çıkarılıyor; bu sütunun korunması isteniyor.
4. `data.drop(columns=columns_to_drop)` ifadesiyle seçilen sütunlar veri çerçevesinden siliniyor.
5. `reset_index()` ile indeksler sıfırlanıyor ve eski indeks sütunu veri çerçevesinden kaldırılıyor. 

Bu adımlar, fazla eksik değer içeren sütunları kaldırarak veri setini daha temiz ve analiz için uygun hale getiriyor.

In [3]:
data_set_null_count = dict(data.isnull().sum())
# ['Ort. Yakıt Tüketimi (Litre)', 'Takasa Uygun', 'Ağır Hasarlı']
columns_to_drop = [column_name for column_name,
                   null_count in data_set_null_count.items() if null_count > 100]

columns_to_drop.remove("Ort. Yakıt Tüketimi (Litre)")
data = data.drop(columns=columns_to_drop).reset_index().drop(columns=["index"])

Bu hücrede, veri setindeki bazı sütunlarda bulunan string (metin) verileri uygun veri tiplerine dönüştürülüyor ve gerektiğinde formatları düzeltiliyor:

1. `data["İlan No"] = data["İlan No"].astype(int)`: Bu satırda, "İlan No" sütunundaki değerler tam sayıya dönüştürülüyor. Bu sütun muhtemelen sayısal bir kimlik numarası olduğu için bu dönüşüm yapılıyor.

2. `data["Ort. Yakıt Tüketimi (Litre)"] = data["Ort. Yakıt Tüketimi (Litre)"].str.replace(',', '.').str.extract(r'(\d.\d?)').astype("Float32")`: Bu işlemde, "Ort. Yakıt Tüketimi (Litre)" sütunundaki virgül (`,`) işaretleri nokta (`.`) ile değiştirilip, ardından ondalıklı sayı formatındaki ilk değeri çekmek için `str.extract` kullanılıyor. Son olarak, sayılar `Float32` türüne dönüştürülüyor. Bu, litre cinsinden yakıt tüketiminin sayısal değeri için yapılır.

3. `data["Motor Hacmi (CC)"] = data["Motor Hacmi (CC)"].str.extract(r'(\d+)').astype("Int64")`: Bu işlem, "Motor Hacmi (CC)" sütunundaki sayıları (motor hacmi) çekiyor ve tam sayı (`Int64`) türüne dönüştürüyor.

4. `data["Motor Gücü (HP)"] = data["Motor Gücü (HP)"].str.extract(r'(\d+)').astype("Int64")`: Aynı şekilde, "Motor Gücü (HP)" sütunundaki sayılar (beygir gücü) çıkarılıyor ve tam sayı türüne dönüştürülüyor.

5. `data["Yakıt Deposu (Litre)"] = data["Yakıt Deposu (Litre)"].str.extract(r'(\d+)').astype("Int64")`: "Yakıt Deposu (Litre)" sütunundaki litre cinsinden yakıt deposu kapasitesi sayısal verilere dönüştürülüyor.

6. `data["Price (TL)"] = data["Price (TL)"].str.extract(r'(\d+)').astype("Int64") * 1000`: "Price (TL)" sütunundaki fiyat verileri önce sayılara dönüştürülüp, ardından bin ile çarpılarak fiyatların tam sayı formatında (TL cinsinden) doğru şekilde hesaplanması sağlanıyor.

7. `data["Kilometre"] = data["Kilometre"].str.extract(r'(\d+)').astype("Int64") * 1000`: "Kilometre" sütunundaki mesafe verileri çıkarılıp, sayıya dönüştürülüyor ve sonra 1000 ile çarpılarak doğru ölçekteki kilometre bilgisi elde ediliyor.

Bu adımlar, veri setindeki string formatındaki sayıları uygun sayısal veri türlerine dönüştürerek analiz için kullanılabilir hale getiriyor.

In [4]:
data["İlan No"] = data["İlan No"].astype(int)

data["Ort. Yakıt Tüketimi (Litre)"] = data["Ort. Yakıt Tüketimi (Litre)"].str.replace(
    ',', '.').str.extract(r'(\d.\d?)').astype("Float32")

data["Motor Hacmi (CC)"] = data["Motor Hacmi (CC)"].str.extract(
    r'(\d+)').astype("Int64")

data["Motor Gücü (HP)"] = data["Motor Gücü (HP)"].str.extract(
    r'(\d+)').astype("Int64")

data["Yakıt Deposu (Litre)"] = data["Yakıt Deposu (Litre)"].str.extract(
    r'(\d+)').astype("Int64")

data["Price (TL)"] = data["Price (TL)"].str.extract(r'(\d+)').astype("Int64") * 1000
data["Kilometre"] = data["Kilometre"].str.extract(r'(\d+)').astype("Int64") * 1000

Temizlenmiş ve ön hazırlık işlemlerinden geçmiş veriseti 'fiat_cars_cleaned.xlsx' dosyasına Excel formatında kaydederiz.

In [ ]:
data.to_excel("./fiat_cars_cleaned.xlsx")